In [ ]:
import pandas as pd
import json

from serlo.mysql import db

In [ ]:
def deep_equal(obj1, obj2):
    return json.dumps(obj1, sort_keys=True) == json.dumps(obj2, sort_keys=True)

#Test
#deep_equal({ "a": { "b": 2 }, "c": [3, 5] }, { "a": { "b": 2 }, "c": [2, 5] })

In [ ]:
def get_keys(obj):
    def append_key(key):
        return [x + [key] for x in get_keys(obj[key])] 
    
    return sum([ append_key(key) if isinstance(obj[key], dict) else [[key]] for key in obj.keys() ], [])

#Test
assert deep_equal(get_keys({ "a": { "b": 2, "r": 4 }, "c": 3 }), [['b', 'a'], ["r", "a"], ["c"]])
assert deep_equal(get_keys({ "a": { "b": { "d": ["a" ]} }, "c": [{"d": 2}] }), [["d", 'b', 'a'], ["c"]])
assert deep_equal(get_keys({}), [])

In [ ]:
def get_keys(obj, stack = []):
    for k, v in obj.items():
        k2 = ([k] if k else []) + stack # don't return empty keys
        if v and isinstance(v, dict):
            for c in get_keys(v, k2):
                yield c
            else: # leaf
                yield k2

def get_values(obj):
    for v in obj.values():
        if not v: continue
        if isinstance(v, dict):
            for c in get_values(v):
                yield c
        else: # leaf
            yield v if isinstance(v, list) else [v]
    
        
def get_value_from_key_array(obj, keys):
    if len(keys) == 0:
        return
    if len(keys) == 1:
        return obj[keys[0]]
    else:
        key = keys.pop(-1)
        return get_value_from_key_array(obj[key], keys)

def leaf_equals(keys, wanted_key):
    return keys[0] == wanted_key

In [ ]:
#Test
#h = {"d": {"k": 3}, "e": 6}
#keys = get_keys(h)
#for key in keys:
#    print(key)
#list(get_keys(h))

In [ ]:
# Hier wird nur der Content geholt. Falls dieser ein Youtube-Video ist, könnte description und eventuell reasoning auch interessant sein.
def select_all_entities():
    return pd.read_sql("""
        SELECT entity.id, value
        FROM entity_revision_field, entity
        where field = "content"
            AND entity.current_revision_id = entity_revision_field.entity_revision_id;
        """, db)

In [ ]:
def get_id(revision):
    return revision["id"]

UNREVISED_REVISIONS_QUERY="""
            {
                uuid(id: %s) {
                    __typename
                    title
                    trashed
                    ... on Applet {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                    ... on Article {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Course {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on CoursePage {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Event {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Exercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on ExerciseGroup {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                cohesive
                            }
                        }
                    }
                    ... on GroupedExercise {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Solution {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                            }
                        }
                    }
                    ... on Video {
                        revisions {
                            nodes {
                                id
                                trashed
                                content
                                url
                            }
                        }
                    }
                }
            }
        """

def get_newest_unrevised_revision(entity_id, bot):
    try:
        response = bot.api_call(query=UNREVISED_REVISIONS_QUERY % entity_id)
    except Exception as e:
        print(e)
        print("Some problem with entity id %s" % entity_id)
        raise e
    revisions = response["uuid"]["revisions"]["nodes"]
    legal_revisions = list(filter(lambda revision: not revision["trashed"], revisions))
    legal_revisions.sort(reverse = True, key = get_id)
    return {
        "revision": legal_revisions[0],
        "metadata": {
            "type": response["uuid"]["__typename"],
            "title": response["uuid"]["title"],
            "trashed": response["uuid"]["trashed"]
        }
    }

def save_revision(entity_id, new_revision, changes, metadata, bot):
    try:
        match metadata["metadata"]["type"]:
            case "Applet":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                    "url": metadata["revision"]["url"]
                }
            case "Video":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                    "url": metadata["revision"]["url"]
                }
            case "Article":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "Course":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "CoursePage":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "Event":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "GroupedExercise":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "Solution":
                custom_input_args = {
                    "title": metadata["metadata"]["title"],
                }
            case "Exercise":
                custom_input_args = {}
            case "ExerciseGroup":
                custom_input_args = {
                    "cohesive": metadata["revision"]["cohesive"],
                }
        entity_args = {
            "subscribeThis": True,
            "subscribeThisByEmail": False,
            "needsReview": True,
            "entityId": entity_id,
            "content": new_revision,
            "changes": changes
        }
        entity_args.update(custom_input_args)
        input_args = create_graphql_input(entity_args)
        query = """
            mutation {
                entity {
                    set%s(input: {
                        %s
                    }) {
                        success
                    }
                }
            }
        """ % (metadata["metadata"]["type"], input_args)
        response = bot.api_call(query)
    except Exception as e:
        print(entity_id)
        raise e
    print("saved")

In [ ]:
def create_graphql_input(args):
    def to_graphql(value):
        if isinstance(value, bool):
            return "true" if value else "false"
        elif isinstance(value, str):
            return repr(value).replace('"','\\\"').replace("'","\"")
        else:
            return value
    
    return "\n".join(["%s: %s" % (k, to_graphql(v)) for k,v in args.items()])

#Test
#print(create_graphql_input({"subscribeThis":True, "foo":3, "content":json.dumps({"a": 2}), "test":"Test"}))